In [71]:
import sys
sys.path.append("..")

In [73]:
from main import nonparametric_test_undirected_vs_bidirected
from main import util
from main import maximal_independent_set
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import networkx as nx
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix


SyntaxError: invalid syntax (nonparametric_test_undirected_vs_bidirected.py, line 1)

In [6]:
def prepare_data(sample, residuals, network, ind_set):
    df = pd.DataFrame()
    
    for node in ind_set:
        row = {'id': int(node)}

        row['value'] = sample[node]
        for k_order in range(1, 3):
            vals = [residuals[i] for i in util.kth_order_neighborhood(network, node, k_order)]
            row[f'{k_order}nb_residual_sum'] = sum(vals)
            row[f'{k_order}nb_count'] = len(vals)

        df = df.append(row, ignore_index=True)
    
    return df

def generate_sample_from_network(network, max_neighbors):
    adjacency_matrix = csr_matrix(nx.adjacency_matrix(nx.from_dict_of_lists(network)))
    c = np.random.uniform(0, 1 / max_neighbors)
    covariance_matrix = c * adjacency_matrix
    np.fill_diagonal(covariance_matrix.toarray(), 1)  # Convert to dense array for fill_diagonal

    # Generate a standard normal sample for each node
    standard_normal_samples = np.random.normal(size=adjacency_matrix.shape[0])

    # Transform the standard normal samples
    sample = covariance_matrix @ standard_normal_samples
    
    return sample, covariance_matrix
    
#     # Generate 1000 samples
#     samples = []
#     for _ in range(100000):
#         # Generate a standard normal sample for each node
#         standard_normal_samples = np.random.normal(size=n_nodes)

#         # Multiply the standard normal samples by the covariance matrix
#         sample = np.dot(covariance_matrix, standard_normal_samples)
#         samples.append(sample)

#     return samples, c

#     return sample

In [55]:
n_nodes = 50000
min_neighbors = 1
max_neighbors = 10

network = util.create_random_network(n_nodes, min_neighbors, max_neighbors)
sample, covariance_matrix  = generate_sample_from_network(network, max_neighbors)

connected_nodes = util.graph_to_edges(network)

/var/folders/2m/s6xvfp8x29j9m9pl6w7jn8hh0000gn/T/ipykernel_68211/106152647.py:18: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adjacency_matrix = csr_matrix(nx.adjacency_matrix(nx.from_dict_of_lists(network)))


In [56]:
ind_set = maximal_independent_set.maximal_n_apart_independent_set(graph=network, n=5, verbose=False)

In [57]:
# Get E[L]
sample = np.array(sample)
mean = np.mean(sample)
residuals = sample - mean

In [58]:
df = prepare_data(sample, residuals, network, ind_set)

/var/folders/2m/s6xvfp8x29j9m9pl6w7jn8hh0000gn/T/ipykernel_68211/106152647.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/var/folders/2m/s6xvfp8x29j9m9pl6w7jn8hh0000gn/T/ipykernel_68211/106152647.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/var/folders/2m/s6xvfp8x29j9m9pl6w7jn8hh0000gn/T/ipykernel_68211/106152647.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/var/folders/2m/s6xvfp8x29j9m9pl6w7jn8hh0000gn/T/ipykernel_68211/106152647.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(

In [59]:
df

,id,value,1nb_residual_sum,1nb_count,2nb_residual_sum,2nb_count
0,0.0,0.021667,-0.016491,5.0,0.153588,31.0
1,1.0,0.020908,0.180308,8.0,0.170159,37.0
2,20481.0,-0.000817,-0.011405,1.0,0.018713,1.0
3,3.0,-0.027848,0.012451,8.0,-0.191875,39.0
4,45060.0,0.006280,-0.046451,1.0,0.015679,6.0
...,...,...,...,...,...,...
741,42986.0,-0.013467,-0.006283,1.0,-0.067031,8.0
742,2032.0,-0.002824,-0.015915,2.0,-0.077845,11.0
743,34800.0,0.005871,0.015893,1.0,0.004473,1.0
744,42990.0,-0.004860,-0.001159,1.0,-0.009997,2.0


In [60]:
def train_and_eval(model, X, y, baseline=False):
    
    if not baseline:
        model.fit(X, y)
        y_pred = model.predict(X)
        print("Coefficients (Slope):", model.coef_)
        print("Intercept:", model.intercept_)
    else:
        y_pred = np.full_like(y, np.mean(y))  # Use the mean value for baseline prediction

    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)

    print("Mean Squared Error:", mse)
    print("R-squared Score:", r2)

In [61]:
# NULL MODEL
X = df[['1nb_residual_sum', '1nb_count']]
y = df['value']
train_and_eval(LinearRegression(), X, y)

Coefficients (Slope): [-0.01150047  0.00016939]
Intercept: -0.00015075957761195803
Mean Squared Error: 5.742154002506747e-05
R-squared Score: 0.0010344642615153221


In [62]:
# ALTERNATIVE MODEL
alt_X = df[['1nb_residual_sum', '1nb_count', '2nb_residual_sum', '2nb_count']]
y = df['value']
train_and_eval(LinearRegression(), alt_X, y)

Coefficients (Slope): [-7.82987342e-03  3.09415518e-05  1.01734754e-01 -1.63339742e-05]
Intercept: 0.000272166049229679
Mean Squared Error: 3.42634431584582e-05
R-squared Score: 0.40391708693128003


In [63]:
train_and_eval(LinearRegression(), X, y, True)

Mean Squared Error: 5.7481002067422304e-05
R-squared Score: 0.0


In [63]:
# Testing the covariance of values between connected units

i, j = connected_nodes[0]

i_vals = []
j_vals = []
for sample in np.array(samples):
    i_vals.append(sample[0][i])
    j_vals.append(sample[0][j])
    
np.cov(i_vals, j_vals)